## TensorFlow Linear

In [7]:
import tensorflow as tf

csvFile = 'pima-indians-diabetes.csv'

In [8]:
learning_rate = 0.000001
epoch = 3
batch_size = 10
filename_queue = tf.train.string_input_producer([csvFile])
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
record_defaults = [[1.], [1.], [1.], [1.], [1.], [1.], [1.], [1.], [1]]

features = tf.decode_csv(value, record_defaults=record_defaults)
X_data = features[:-1]
Y_data = features[-1:]
batch_xs, label = tf.train.shuffle_batch(tensors=[X_data, Y_data], batch_size=batch_size,
                                         num_threads=1,
                                         capacity=5000,
                                         min_after_dequeue=100)
label = tf.one_hot(label, depth=2) # 0 => [1,0] 1 => [0,1]
label = tf.reshape(label,[-1,2])
features = len(X_data)
classes = label.shape[1]
print(features, classes)

8 2


In [9]:
X = tf.placeholder(tf.float32, [None, features], name="input") # 1,8
Y = tf.placeholder(tf.float32, [None, classes], name="output")  # 1,1

W = tf.Variable(tf.zeros([features, classes]), name='weight')    # (8,1)
b = tf.Variable(tf.zeros([classes]), name='bias')                # (1)

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)                          # (1,8) * (8,1) + (1)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), reduction_indices=1))              # 

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

In [10]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(tf.global_variables_initializer())
    
    for i in range(epoch):
        for step in range(501):
            opt, cc = sess.run([optimizer,cost], feed_dict={X:batch_xs.eval(), Y:label.eval()})
    #         print(batch_xs.eval(),label.eval())
            if step%20 ==0:
                atemp = accuracy.eval({X:batch_xs.eval(), Y:label.eval()})
                print(step, cc, atemp)
    
    coord.request_stop()
    coord.join(threads)

0 0.693147 0.7
20 0.681341 0.7
40 0.664413 0.6
60 0.588754 0.5
80 0.77681 0.6
100 0.651716 0.7
120 0.805735 0.5
140 0.697034 0.7
160 0.647502 0.6
180 0.579476 0.7
200 0.613562 0.8
220 0.791158 0.7
240 0.659067 0.6
260 0.659761 0.6
280 0.717329 0.4
300 0.549611 0.6
320 0.63116 0.5
340 0.679 0.9
360 0.556337 0.7
380 0.836551 0.8
400 0.657171 0.8
420 0.595872 0.6
440 0.587748 0.9
460 0.962484 0.6
480 0.837184 1.0
500 0.591352 0.6
0 0.62702 0.8
20 0.848096 0.4
40 0.676267 0.5
60 0.725188 0.5
80 0.668939 0.6
100 0.580466 0.9
120 0.606377 0.6
140 0.601034 0.9
160 0.723176 0.7
180 0.586127 0.6
200 0.750789 0.5
220 0.572033 0.6
240 0.648537 0.8
260 0.664084 0.6
280 0.819327 0.8
300 0.771088 0.6
320 0.68254 0.8
340 0.537846 0.5
360 0.548653 0.7
380 0.700213 0.6
400 0.608572 0.6
420 0.86783 0.6
440 0.775928 1.0
460 0.61103 0.6
480 0.479484 0.6
500 0.664261 0.5
0 0.690591 0.7
20 0.660671 0.6
40 0.722211 0.8
60 0.638699 0.5
80 0.671803 0.6
100 0.55085 0.6
120 0.546017 0.6
140 0.854106 0.6
160 0.51